# code
- make dataset
- train
- evaluate

In [198]:
# reference: https://github.com/pengyuchen/PyTorch-Batch-Seq2seq/blob/master/seq2seq_translation_tutorial.py

import unicodedata
import re
import torch
from torch.autograd import Variable
import time
import math
import random
SOS_token = 0
EOS_token = 1
PAD_token = 2
use_cuda = torch.cuda.is_available()

class saveLang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS", 2: 'PAD'}
        self.n_words = 3  # Count SOS and EOS

    def update_voc_dict(self, sentence):
        for word in sentence.split(' '):
            if word not in self.word2index:
                self.word2index[word] = self.n_words
                self.word2count[word] = 1
                self.index2word[self.n_words] = word
                self.n_words += 1
            else:
                self.word2count[word] += 1

def _make_pair(lang1, lang2, reverse=False):
    #lines = open('%s-%s.txt' % (lang1, lang2), encoding='utf-8').read().strip().split('\n')
    # toy dataset
    lines = ['j a l f h z z\tz z h f l a j', 'j y w a r e d v\tv d e r a w y j', 'o c q u c w\tw c u q c o', 'l q\tq l', 'b z g b c\tc b g z b', 'p l\tl p', 'p o n j d u u c r p\tp r c u u d j n o p', 'm m l p r j d\td j r p l m m', 'k b s f l u n p\tp n u l f s b k', 'r a o i d d s a k\tk a s d d i o a r', 'u n p r q\tq r p n u', 'w\tw', 'h h x i d c z w j p\tp j w z c d i x h h', 'u\tu', 'f\tf', 'o o l u w h z u j\tj u z h w u l o o', 'h v k p y n d j a\ta j d n y p k v h', 'j f u v u\tu v u f j', 'l n m u\tu m n l', 'j\tj', 'w h\th w', 's l r w z r l y\ty l r z w r l s', 'r c b r u k x i q\tq i x k u r b c r', 'u u d q t m y i c v\tv c i y m t q d u u', 'w i m v v r\tr v v m i w', 'a f z e\te z f a', 'y x k x c\tc x k x y', 'f k z n s c v b b\tb b v c s n z k f', 'o g\tg o', 'j c m d b\tb d m c j', 'p t k v f v a f\tf a v f v k t p', 'b t\tt b', 'a z q l i l\tl i l q z a', 'm a q a t\tt a q a m', 'd e l q t t\tt t q l e d', 'g g h x j i x\tx i j x h g g', 'a u a y l\tl y a u a', 'g c g o h q i c\tc i q h o g c g', 'w f h c a z l\tl z a c h f w', 'e\te', 'y h\th y', 'a t i t m k r q\tq r k m t i t a', 'l s\ts l', 'l t u v n n v\tv n n v u t l', 'b x c c t s x w r q\tq r w x s t c c x b', 'c o o x j d d r z\tz r d d j x o o c', 'e g z k q i d\td i q k z g e', 'v n u o\to u n v', 'u f g i d c q x g\tg x q c d i g f u', 'x z p r j z l y\ty l z j r p z x', 'b h c\tc h b', 'a u\tu a', 'y v z q j p b l f s\ts f l b p j q z v y', 'm q f j u v h p d u\tu d p h v u j f q m', 'q c f b p k w v p\tp v w k p b f c q', 't u\tu t', 'q y v i g x j s\ts j x g i v y q', 'b r u t m m\tm m t u r b', 'c b n c i g b v\tv b g i c n b c', 'd j d v a\ta v d j d', 'e v\tv e', 'd q n r y h i\ti h y r n q d', 'q i x d e f c\tc f e d x i q', 'f g p\tp g f', 'f b v w\tw v b f', 'k f n u s u q\tq u s u n f k', 'p l c f h i\ti h f c l p', 'c m p\tp m c', 'f k r k p d c z\tz c d p k r k f', 'f y s f n m d x i x\tx i x d m n f s y f', 'f z g f h a q d w\tw d q a h f g z f', 'l l z g o j j j v\tv j j j o g z l l', 'j r\tr j', 'a t e f a o y c\tc y o a f e t a', 'z i m i u r v n t g\tg t n v r u i m i z', 'i p c v y q\tq y v c p i', 'a r s t n f l i\ti l f n t s r a', 'v i r w u l h n d\td n h l u w r i v', 'h c r m c h x j d b\tb d j x h c m r c h', 'u c t\tt c u', 'n\tn', 'i s q x i a u s m u\tu m s u a i x q s i', 't r d d\td d r t', 'l t k v n j k\tk j n v k t l', 'i q\tq i', 'e b y a e k r f m\tm f r k e a y b e', 'l y o x a\ta x o y l', 'f m c v s m u q\tq u m s v c m f', 'e w y c l y q\tq y l c y w e', 'b w\tw b', 'j j f a a n\tn a a f j j', 'a u c z f e\te f z c u a', 'k m a j\tj a m k', 'm t j p a\ta p j t m', 'j n p s c\tc s p n j', 'g n n t x s o\to s x t n n g', 'q f h p e h\th e p h f q', 'q j x\tx j q', 'b s\ts b', 'm h g i\ti g h m']
    pairs = [[s for s in l.split('\t')] for l in lines]
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = saveLang(lang2)
        output_lang = saveLang(lang1)
    else:
        input_lang = saveLang(lang1)
        output_lang = saveLang(lang2)
    return input_lang, output_lang, pairs

def make_dataset(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = _make_pair(lang1, lang2, reverse)
    for pair in pairs:
        input_lang.update_voc_dict(pair[0])
        output_lang.update_voc_dict(pair[1])
    return input_lang, output_lang, pairs

def _sen2idx(lang, sentence, time_step):
    idxs = [lang.word2index[word] for word in sentence.split(' ')][:time_step]
    idxs.append(EOS_token)
    idxs.extend([PAD_token] * (time_step - len(idxs)))
    result = torch.LongTensor(idxs)
    if use_cuda:
        return result.cuda()
    else:
        return result

def pairs2idx(input_lang, output_lang, pairs, time_step):
    dataset = []
    for pair in pairs:
        input_variable = _sen2idx(input_lang, pair[0], time_step)
        target_variable = _sen2idx(output_lang, pair[1], time_step)
        dataset.append((input_variable, target_variable))
    return dataset

# time step should be long enough
time_step = 50
# use_cuda = torch.cuda.is_available()

input_lang, output_lang, pairs = make_dataset('eng', 'kor',  reverse=False)
pairs = pairs2idx(input_lang, output_lang, pairs, time_step)
print(len(dataset))

100


In [140]:
class Encoder(nn.Module):
    def __init__(self, voc_size, embed_size, hidden_size):
        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(voc_size, embed_size)
        self.embedding.weight.data.copy_(torch.eye(voc_size, embed_size))
        self.embedding.weight.requires_grad = True
        
        self.gru = nn.GRU(embed_size, hidden_size)

    def forward(self, input_data, batch_size, hidden):
        embed = self.embedding(input_data).view(1, batch_size, self.embed_size)
        output = embed
        # reference: https://discuss.pytorch.org/t/clarification-regarding-the-return-of-nn-gru/47363
        output, hidden = self.gru(output, hidden)
        return output, hidden
    
    def first_hidden(self, batch_size):
        val =  Variable(torch.zeros(1, batch_size, self.hidden_size))
        if use_cuda:
            return val.cuda()
        else:
            return val

In [233]:
class Decoder(nn.Module):
    def __init__(self, voc_size, embed_size, hidden_size,):
        super(Decoder, self).__init__()
        
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(voc_size, embed_size)
        self.embedding.weight.data.copy_(torch.eye(voc_size, embed_size))
        self.embedding.weight.requires_grad = True
        
        self.gru = nn.GRU(embed_size, hidden_size)
        self.out = nn.Linear(hidden_size, voc_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, input_data, batch_size, hidden):
        embed = self.embedding(input_data).view(1, batch_size, self.embed_size)
        output = embed
        output, hidden = self.gru(output, hidden)
        output = self.out(output) # shape: [1, batch_size, voc_size]
        output = self.softmax(output[0])
        return output, hidden
    
    def first_hidden(self, batch_size):
        val =  Variable(torch.zeros(1, batch_size, self.hidden_size))
        if use_cuda:
            return val.cuda()
        else:
            return val

In [234]:
SOS_token = 0
EOS_token = 1
PAD_token = 2

teacher_forcing_ratio = 0.5
embed_size = 100
hidden_size = 30 #a-z+SOS+EOS+PAD
batch_size = 20
epochs = 1

encoder = Encoder(voc_size=input_lang.n_words, embed_size=embed_size, hidden_size=hidden_size)
decoder = Decoder(voc_size=input_lang.n_words, embed_size=embed_size, hidden_size=hidden_size)

use_cuda = False

if use_cuda:
    encoder = encoder.cuda()
    decoder = decoder.cuda()

In [235]:
def batch_train(input_variable, target_variable, encoder, decoder, encoder_optim, decoder_optim, criterion, time_step, teacher_forcing_ratio):
    
    # encoder part
    batch_size = input_variable.size()[0]
    encoder_hidden = encoder.first_hidden(batch_size)
    
    # input_variable: [batch_size, n_time_step]
    # input_variable.transpose: [n_time_step, batch_size]
    input_variable = Variable(input_variable.transpose(0, 1))
    encoder_optim.zero_grad()
    en_time_step = input_variable.size()[0]
    
    loss = 0
    encoder_outputs = Variable(torch.zeros(time_step, batch_size, encoder.hidden_size))
    for i in range(en_time_step):
        en_output, en_hidden = encoder(input_variable[i], batch_size, encoder_hidden)
        encoder_outputs[i] = en_output # ???????
    
    # decoder part
    # first input: SOS token
    target_variable = Variable(target_variable.transpose(0, 1))
    decoder_optim.zero_grad()
    de_time_step = target_variable.size()[0]
    
    decoder_input = Variable(torch.LongTensor([SOS_token] * batch_size))
    de_hidden = en_hidden
    
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    
    if use_teacher_forcing:
        for i in range(de_time_step):
            de_out, de_hidden = decoder(decoder_input, batch_size, de_hidden)
            loss += criterion(de_out, target_variable[i])
            decoder_input = target_variable[i]
            
    else:
        for i in range(de_time_step):
            de_out, de_hidden = decoder(decoder_input, batch_size, de_hidden)
            loss += criterion(de_out, target_variable[i])
            
            topv, topi = de_out.data.topk(1) # topi: voc_idx
            decoder_input = Variable(torch.flatten(topi)) 
            decoder_input  
    
    # update weights after finish forward 1 batch
    loss.backward()
    
    encoder_optim.step()
    decoder_optim.step()
    
    return loss.data / de_time_step

In [238]:
def train(encoder, decoder, epochs, train_loader, test_loader, time_step, learning_rate=0.01, teacher_forcing_ratio=0.5):
        encoder_optim = optim.SGD(filter(lambda x: x.requires_grad, encoder.parameters()), lr=learning_rate)
        decoder_optim = optim.SGD(filter(lambda x: x.requires_grad, decoder.parameters()), lr=learning_rate)
        
        criterion = nn.NLLLoss()
        for epoch in range(epochs):
            print_loss_total = 0
            for batch_x, batch_y in train_loader:
                loss = batch_train(batch_x, batch_y, encoder, decoder, encoder_optim, decoder_optim, criterion, time_step, teacher_forcing_ratio)
                print_loss_total += loss
            print('epochs: '+str(epoch))
            print('total loss: '+str(print_loss_total))
            print()

In [240]:
train_num = int(len(pairs)*0.9)
train_loader = torch.utils.data.DataLoader(pairs[:train_num], 
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(pairs[train_num:], 
    batch_size=batch_size, shuffle=True)

epochs = 20
train(encoder, decoder, epochs, train_loader, test_loader, time_step)

epochs: 0
total loss: tensor(3.3093)

epochs: 1
total loss: tensor(3.5457)

epochs: 2
total loss: tensor(3.3186)

epochs: 3
total loss: tensor(3.0699)

epochs: 4
total loss: tensor(3.2578)

epochs: 5
total loss: tensor(3.1709)

epochs: 6
total loss: tensor(3.1182)

epochs: 7
total loss: tensor(2.9570)

epochs: 8
total loss: tensor(2.8541)

epochs: 9
total loss: tensor(2.7336)

epochs: 10
total loss: tensor(2.8898)

epochs: 11
total loss: tensor(2.7830)

epochs: 12
total loss: tensor(2.6468)

epochs: 13
total loss: tensor(2.8635)

epochs: 14
total loss: tensor(2.6827)

epochs: 15
total loss: tensor(2.9227)

epochs: 16
total loss: tensor(2.8652)

epochs: 17
total loss: tensor(2.7276)

epochs: 18
total loss: tensor(2.7731)

epochs: 19
total loss: tensor(2.7706)

